In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn import datasets, metrics
import math
training1 = pd.read_csv('https://raw.githubusercontent.com/EmZhu/HiggsBoson/master/train_jet0_final.csv')
training2 = pd.read_csv('https://raw.githubusercontent.com/EmZhu/HiggsBoson/master/train_jet1_final.csv')
training3 = pd.read_csv('https://raw.githubusercontent.com/EmZhu/HiggsBoson/master/train_jet2.3_final.csv')
sub_training1=training1.drop('EventId',1).drop('Label',1).drop('Weight',1)
sub_training2=training2.drop('EventId',1).drop('Label',1).drop('Weight',1)
sub_training3=training3.drop('EventId',1).drop('Label',1).drop('Weight',1)
training_set=[sub_training1,sub_training2,sub_training3]
training1.ix[training1.Label=='s','Label']=0
training1.ix[training1.Label=='b','Label']=1
training2.ix[training2.Label=='s','Label']=0
training2.ix[training2.Label=='b','Label']=1
training3.ix[training3.Label=='s','Label']=0
training3.ix[training3.Label=='b','Label']=1
for i in range(len(training_set)):
    length=len(training_set[i])
    subset_length=length/10
    exec("training_val"+str(i) + " = []")
    for j in range(9):
        temp_set=training_set[i][(subset_length*j):(subset_length*(j+1))]
        exec("training_val"+str(i) + ".append(temp_set)")
    temp_set=training_set[i][(subset_length*9):]
    exec("training_val"+str(i) + ".append(temp_set)")
sub_test1=training1['Label']
sub_test2=training2['Label']
sub_test3=training3['Label']
testing_set=[sub_test1,sub_test2,sub_test3]
for i in range(len(testing_set)):
    length=len(testing_set[i])
    subset_length=length/10
    exec("test_val"+str(i) + " = []")
    for j in range(9):
        temp_set=testing_set[i][(subset_length*j):(subset_length*(j+1))]
        exec("test_val"+str(i) + ".append(temp_set)")
    temp_set=testing_set[i][(subset_length*9):]
    exec("test_val"+str(i) + ".append(temp_set)")
sub_weight1=training1['Weight']
sub_weight2=training2['Weight']
sub_weight3=training3['Weight']
weight_set=[sub_weight1,sub_weight2,sub_weight3]
for i in range(len(weight_set)):
    length=len(weight_set[i])
    subset_length=length/10
    exec("weight_val"+str(i) + " = []")
    for j in range(9):
        temp_set=weight_set[i][(subset_length*j):(subset_length*(j+1))]
        exec("weight_val"+str(i) + ".append(temp_set)")
    temp_set=weight_set[i][(subset_length*9):]
    exec("weight_val"+str(i) + ".append(temp_set)")
eval_x=[training_val0,training_val1,training_val2]
eval_y=[test_val0,test_val1,test_val2]
eval_w=[weight_val0,weight_val1,weight_val2]
def AMS(real,pred,weight):
    pred_s_ind=np.array(pred==0)
    real_s_ind=np.array(real==0)
    real_b_ind=np.array(real==1)
    
    s=np.sum(weight[pred_s_ind & real_s_ind])
    b=np.sum(weight[pred_s_ind & real_b_ind])
    
    b_tau=10
    ans=math.sqrt(2*((s+b+b_tau)*math.log(1+s/(b+b_tau))-s))
    return (ans)
sumarray=np.load('sumarray.npy')

mean_array=np.apply_along_axis(lambda x: np.mean(x), 1, sumarray)
mean_array1=mean_array[range(11)]
mean_array2=mean_array[range(11,22)]
mean_array3=mean_array[range(22,33)]

units_for_1=np.argmax(mean_array1)+10
units_for_2=np.argmax(mean_array2)+10
units_for_3=np.argmax(mean_array3)+10



In [4]:
optimizer=['GradientDescentOptimizer','AdadeltaOptimizer','AdagradOptimizer','AdamOptimizer','FtrlOptimizer','RMSPropOptimizer']
max_mean=[units_for_1,units_for_2,units_for_3]
learning_rate=['0.001','0.002','0.003','0.004','0.005','0.006','0.007','0.008','0.009','0.01']

learning_model=[]
for learn in learning_rate:
    num_of_units=[]
    for k in range(len(eval_x)):
        #set_group2=[]
        i=max_mean[k]
        temp_units=[]
        #temp_units2=[]
        for l in optimizer:
            optimizer_group=[]
            for j in range(10):
                print (k,j,l,learn)
                exec('classifier=tf.contrib.learn.DNNClassifier(hidden_units=[i], n_classes=2,optimizer=tf.train.'+l+'(learning_rate='+learn+'))')
                validation_x=eval_x[k][j]
                validation_y=eval_y[k][j]
                validation_w=eval_w[k][j]
                train_x=pd.concat(eval_x[k][:j]+eval_x[k][j+1:])
                train_y=pd.concat(eval_y[k][:j]+eval_y[k][j+1:])
                classifier.fit(x=train_x.values, y=np.array(train_y, dtype=np.int), steps=200)
                #temp_units2.append(classifier)
                pred=classifier.predict(validation_x.values)
                accuracy=AMS(validation_y.values,pred,validation_w.values)
                print (accuracy)
                optimizer_group.append(accuracy)
            temp_units.append(optimizer_group)
        num_of_units.append(temp_units)
    learning_model.append(num_of_units)



Instructions for updating:
Pass `tf.contrib.learn.infer_real_valued_columns_from_input(x)` or `tf.contrib.learn.infer_real_valued_columns_from_input_fn(input_fn)` as `feature_columns`, where `x` or `input_fn` is your argument to `fit`, `evaluate`, or `predict`.
Instructions for updating:
Pass `tf.contrib.learn.infer_real_valued_columns_from_input(x)` or `tf.contrib.learn.infer_real_valued_columns_from_input_fn(input_fn)` as `feature_columns`, where `x` or `input_fn` is your argument to `fit`, `evaluate`, or `predict`.


(0, 0, 'GradientDescentOptimizer', '0.001')
0.334027217212

Instructions for updating:
Pass `tf.contrib.learn.infer_real_valued_columns_from_input(x)` or `tf.contrib.learn.infer_real_valued_columns_from_input_fn(input_fn)` as `feature_columns`, where `x` or `input_fn` is your argument to `fit`, `evaluate`, or `predict`.
Instructions for updating:
Pass `tf.contrib.learn.infer_real_valued_columns_from_input(x)` or `tf.contrib.learn.infer_real_valued_columns_from_input_fn(input_fn)` as `feature_columns`, where `x` or `input_fn` is your argument to `fit`, `evaluate`, or `predict`.



(0, 1, 'GradientDescentOptimizer', '0.001')


KeyboardInterrupt: 